In [21]:
import pandas as pd
from scipy.stats import norm

import imperial_model
from imperial_model import ImperialModel

import case_distribution_model
from case_distribution_model import CaseDistributionModel

from historian_model import HistorianModel
from arima_residuals_model import ArimaResidualsModel
from benchmark_model import BenchmarkModel
from arima_model import ArimaModel

In [22]:
arima_residuals_model = ArimaResidualsModel()
arima_residuals_preds = arima_residuals_model.get_predictions_for_all_counties("2020-05-24", 18)

In [27]:
arima_model = ArimaModel()
arima_preds = arima_model.get_predictions_for_all_counties("2020-05-24", 18)

In [ ]:
arima_preds[1045]

In [29]:
benchmark_model = BenchmarkModel()
benchmark_preds = benchmark_model.predict_all_counties("2020-06-07", "2020-05-24")

In [38]:
historian_model = HistorianModel("2020-05-24")

fips_list = imperial_model.get_top_cumuluative_death_counties(100, "2020-05-24")[30:]
historian_preds = {}
for fips in fips_list:
    historian_preds[fips] = historian_model.get_predictions(fips, 18)

In [33]:
case_model = CaseDistributionModel()
case_counties = imperial_model.get_top_cumuluative_death_counties(100, "2020-05-24")

fit = case_model.train(case_counties, "2020-05-24", 14, 50, 400, 4)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_09a2e01a82b24cf2e68c80bc9db06884 NOW.


Compiling model...
Training model...


To run all diagnostics call pystan.check_hmc_diagnostics(fit)


Training compelted.


In [34]:
case_model_preds = case_model.get_predictions_map(fit)

In [ ]:
imp_model = ImperialModel()
imp_counties = imperial_model.get_top_cumuluative_death_counties(100, "2020-05-09")
fit = imp_model.train_imperial_model(imp_counties, "2020-05-09", 16, "2020-05-09", 300, 4)

In [ ]:
key = pd.read_csv("../data/us/processing_data/fips_key.csv", encoding='latin-1')
all_fips = key["FIPS"].tolist()

imperial_preds = imp_model.get_imperial_predictions_map(fit)

In [36]:
sample_submission = pd.read_csv("../sample_submission.csv")

#dates_of_interest = ['2020-05-10', '2020-05-11', '2020-05-12', '2020-05-13', '2020-05-14', '2020-05-15', '2020-05-16', '2020-05-17', '2020-05-18', '2020-05-19', '2020-05-20', '2020-05-21', '2020-05-22', '2020-05-23']
dates_of_interest = ['2020-05-25', '2020-05-26', '2020-05-27', '2020-05-28', '2020-05-29', '2020-05-30', '2020-05-31', '2020-06-01', '2020-06-02', '2020-06-03', '2020-06-04', '2020-06-05', '2020-06-06', '2020-06-07']

def get_id_list():
    return sample_submission["id"].values

def extract_date_from_id(row_id):
    split = row_id.split('-')
    return '-'.join(split[:-1])

def extract_fips_from_id(row_id):
    return row_id.split('-')[-1]

def generate_quantiles(value, err):
    if err == 0:
        return [value] * 9
    
    quantiles = []
    for quantile in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
        quantiles.append(norm.ppf(quantile, loc=value, scale=err))

    return quantiles

def create_submission_from_map(preds, output_file):
    lists = []
    for row_id in get_id_list():
        date = extract_date_from_id(row_id)
        fips = int(extract_fips_from_id(row_id))
        
        if not fips in preds:
            lst = [row_id] + ["%.2f" % 0.00] * 9
            lists.append(lst)
            continue
            
        if not date in dates_of_interest:
            lst = [row_id] + ["%.2f" % 0.00] * 9
            lists.append(lst)
            continue
            
        date_index = dates_of_interest.index(date)
        quantiles = generate_quantiles(preds[fips][date_index], preds[fips][date_index] * 0.6)
        lst = [row_id]
        for q in quantiles:
            if q == "nan":
                lst.append("%.2f" % 0.00)
            elif q < 0:
                lst.append("%.2f" % 0.00)
            else:
                lst.append("%.2f" % q)
        
        lists.append(lst)
    
    df = pd.DataFrame(lists, columns=sample_submission.columns)
    df.to_csv(output_file, index=False, sep=',')

In [39]:
create_submission_from_map(historian_preds, "submissions/historian_model_5-24.csv")

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:46: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
